# Measure time to run the algorithm
$\phi=0 => \mathrm{Step\; 2}$

$\phi=1 => \mathrm{Step\; 1}$ 

In [1]:
#Import the relevant modules
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import time
import networkx as nx

import OpinionGraph
import OpinionAlgorithm

In [2]:
n = 3200
m = 6400
gamma = 10
n_opinion = int(n/gamma)
G = OpinionGraph.CreateRandom(n, m, n_opinion)

In [3]:
#Create random graph
%timeit G = OpinionGraph.CreateRandom(n, m, n_opinion)

76 ms ± 6.62 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Step 1

In [4]:
node_i = 1029
bool_step = True

In [5]:
%timeit OpinionAlgorithm.OneIteration(G, node_i, bool_step)

351 µs ± 44.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [6]:
%timeit OpinionAlgorithm.Step1(G, node_i, False)

309 µs ± 11.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Let's decompose it

In [7]:
#take opinion of node_i
opinion_gi = G.nodes[node_i]['opinion']

#Compute neighbours of node_i (can have itself, and many occurence of same neighbour)
neighbors = list(G.neighbors(node_i))

#Select random neighbor. This aquaintance will be deleted. (tend to delete more easily the ''mulit-aquaintances'
#or multi-self-loop')
node_j = int(np.random.choice(neighbors, 1))

#Compute all nodes with opinion of node_i
nodes_with_gi = [n for n, attr in G.nodes(data=True) if attr['opinion']==opinion_gi]

#Select randomly the new aquaintance
node_j_prime = int(np.random.choice(nodes_with_gi, 1))

#G.remove_edge(node_i, node_j)
#G.add_edge(node_i, node_j_prime)

In [8]:
%timeit opinion_gi = G.nodes[node_i]['opinion']

715 ns ± 9.89 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [9]:
%timeit neighbors = list(G.neighbors(node_i))

559 ns ± 24.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [10]:
%timeit node_j = int(np.random.choice(neighbors, 1))

11.7 µs ± 684 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [11]:
%timeit nodes_with_gi = [n for n, attr in G.nodes(data=True) if attr['opinion']==opinion_gi]

370 µs ± 52 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [12]:
%timeit G.nodes(data=True)

1.06 µs ± 10.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [13]:
%timeit node_j_prime = int(np.random.choice(nodes_with_gi, 1))

11.2 µs ± 903 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Step 1 takes ~300 $\mu s$ and is limitted by `nodes_with_gi = [n for n, attr in G.nodes(data=True) if attr['opinion']==opinion_gi]` but I don't see how to make this step faster

### Step 2

In [14]:
bool_step = False

In [15]:
%timeit OpinionAlgorithm.OneIteration(G, node_i, bool_step)

22.2 µs ± 2.35 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [16]:
%timeit OpinionAlgorithm.Step2(G, node_i, False)

16.4 µs ± 929 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Let's decompose it

In [17]:
neighbors = list(G.neighbors(node_i))

#Select random neighbor.
node_j = int(np.random.choice(neighbors, 1))

#Change opinion of node_i to the one of node_j
G.node[node_i]['opinion'] = G.node[node_j]['opinion']

In [18]:
%timeit neighbors = list(G.neighbors(node_i))

570 ns ± 8.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [19]:
%timeit node_j = int(np.random.choice(neighbors, 1))

10.3 µs ± 379 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [20]:
%timeit G.node[node_i]['opinion'] = G.node[node_j]['opinion']

1.56 µs ± 69.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Step 2 takes ~18 $\mu s$ and is limitted by `node_j = int(np.random.choice(neighbors, 1))` but I don't see how to make this step faster

### Overal computation time
If $\phi=1$, for a graph with $N =3200$, we expect $\tau=24$ where $\tau$ is the number of updates per vertex needed to reach consensus. Thus, would need $24\cdot3200=76800$ steps.

In [21]:
OneStepTime = 300e-6/2
NStep = 76800
Total_time = OneStepTime*NStep

This should take (in sec)

In [22]:
Total_time

11.52

If we want 10000 iterations, that's (in hours)

In [23]:
Total_time*10000/3600

32.0

which is long ... but somehow ok ... Or maybe not because we want a lot of those... __How can we make it faster ?__

### But when we compute consensus, seems way longer ...

In [24]:
%timeit OpinionGraph.ConsensusState(G)

7.25 ms ± 300 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [25]:
%timeit OpinionGraph.Components(G)

4.46 ms ± 226 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Apparently the evaluation of consensus state is the bottleneck ...

Connected components

In [26]:
%timeit nx.connected_components(G)

1.35 µs ± 56.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [27]:
%timeit list(nx.connected_components(G))

4.36 ms ± 426 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [28]:
%timeit [a for a in nx.connected_components(G)]

4.28 ms ± 154 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Connected components subgraphs

In [29]:
%timeit nx.connected_component_subgraphs(G)

1.32 µs ± 17.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [30]:
%timeit list(nx.connected_component_subgraphs(G))

799 ms ± 43.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Node attributes

In [31]:
%timeit nx.get_node_attributes(G, 'opinion')

1.4 ms ± 75.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [32]:
%timeit nx.get_node_attributes(G, 'opinion').values()

1.36 ms ± 68.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [33]:
%timeit np.array(list(nx.get_node_attributes(G, 'opinion').values()))

1.62 ms ± 16.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


How can we make this faster ? It this due to the multigraph? 

In [34]:
%timeit graph = nx.gnm_random_graph(n, m) 

23.6 ms ± 365 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


First, this is a lot faster (3 times) than our function

In [35]:
graph = nx.gnm_random_graph(n, m) 

In [36]:
%timeit OpinionGraph.Components(graph)

4.4 ms ± 113 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


But does not appear to make a difference... __How can we do this consensus state evaluation faster?__ We could evaluate only every x iterations (not great to find tau but for the rest should be fine)